In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

titanic_data = Training data set         
titanic_test_data = Testing data set

In [5]:
 titanic_data = pd.read_csv('/content/drive/My Drive/titanic.zip (Unzipped Files)/train.csv')
 titanic_data.head()
 titanic_test_data = pd.read_csv('/content/drive/My Drive/titanic.zip (Unzipped Files)/test.csv')
 titanic_test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# ***Handling missing values of*** **Age** 

In [6]:
titanic_data['Title'] = titanic_data['Name'].str.extract('([A-Za-z]+)\.',expand=True)
titanic_test_data['Title'] = titanic_test_data.Name.str.extract('([A-Za-z]+)\.', expand=True)
titanic_test_data.loc[titanic_test_data.Title.isnull()]
title_map = {'Capt': 'Mr', 'Col': 'Mr', 'Rev': 'Mr', 'Countess': 'Mrs', 'Don': 'Mr', 'Jonkheer': 'Mr',
             'Lady': 'Mrs', 'Major': 'Mr', 'Mlle': 'Miss', 'Ms': 'Miss','Mme': 'Mrs', 'Sir': 'Mr', 'Dona': 'Mrs'}

titanic_data.replace(title_map, inplace=True)
titanic_test_data.replace(title_map, inplace=True)

title_cat = ['Mrs', 'Miss', 'Mr', 'Master', 'Dr']
for title in range(len(title_cat)):
  train_age_title_mean = titanic_data.groupby('Title')['Age'].mean()[title_cat[title]]
  test_age_title_mean = titanic_test_data.groupby('Title')['Age'].mean()[title_cat[title]]
  
  titanic_data.loc[(titanic_data.Age.isnull() & titanic_data.Title.str.contains(title_cat[title])), 'Age'] = train_age_title_mean
  titanic_test_data.loc[(titanic_test_data.Age.isnull() & titanic_test_data.Title.str.contains(title_cat[title])), 'Age'] = test_age_title_mean
  

## **Handling Cabin missing Values**

In [7]:
titanic_data['Deck'] = titanic_data['Cabin'].str.extract('([A-Za-z]+)\d',expand=True)
titanic_test_data['Deck'] = titanic_test_data['Cabin'].str.extract('([A-Za-z]+)\d',expand=True)

titanic_data.groupby('Deck').count()
titanic_test_data.groupby('Deck').count()

titanic_data.loc[titanic_data.Deck.isnull(), 'Deck'] = 'NA'
titanic_test_data.loc[titanic_test_data.Deck.isnull(), 'Deck'] = 'NA'

## **Handling Fare missing values**

In [9]:
titanic_test_data.Fare.fillna(titanic_test_data.Fare.mean(), inplace=True)

## ***Handling Embarked Missing Value***

In [10]:
titanic_data.Embarked.fillna(titanic_data.Embarked.mode, inplace=True)

## ***Handling Is alone and Fam***




In [16]:
titanic_data['Fam'] = titanic_data.SibSp + titanic_data.Parch
titanic_test_data['Fam'] = titanic_test_data.SibSp + titanic_test_data.Parch

# titanic_data['is_alone'] = 1
# titanic_data.loc[titanic_data.Fam > 0 , 'is_alone'] = 0

# titanic_test_data['is_alone'] = 1
# titanic_test_data.loc[titanic_test_data.Fam > 0, 'is_alone'] = 0

# ***Setting X_train, X_test and y_train***


In [19]:
X_train = titanic_data.drop(['Survived', 'Name', 'Ticket', 'Cabin', 'Title', 'Deck'], axis=1)
cols = list(X_train.columns.values)
y_train = titanic_data['Survived']
X_test = titanic_test_data.drop(['Name', 'Ticket', 'Cabin', 'Title', 'Deck'], axis=1)
X_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Fam,is_alone
0,1,3,male,22.0,1,0,7.2500,S,1,0
1,2,1,female,38.0,1,0,71.2833,C,1,0
2,3,3,female,26.0,0,0,7.9250,S,0,1
3,4,1,female,35.0,1,0,53.1000,S,1,0
4,5,3,male,35.0,0,0,8.0500,S,0,1


In [20]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Embarked     891 non-null    object 
 8   Fam          891 non-null    int64  
 9   is_alone     891 non-null    int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 69.7+ KB


## ***Encoding***

In [21]:
label_encoder = LabelEncoder()
X_train['Sex'] = label_encoder.fit_transform(X_train['Sex'])
X_train['Pclass'] = label_encoder.fit_transform(X_train['Pclass'])
X_train['Embarked'] = label_encoder.fit_transform(X_train['Embarked'].astype('str'))

X_test['Sex'] = label_encoder.fit_transform(X_test['Sex'])
X_test['Pclass'] = label_encoder.fit_transform(X_test['Pclass'])
X_test['Embarked'] = label_encoder.fit_transform(X_test['Embarked'].astype('str'))
X_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Fam,is_alone
0,1,2,1,22.0,1,0,7.2500,3,1,0
1,2,0,0,38.0,1,0,71.2833,1,1,0
2,3,2,0,26.0,0,0,7.9250,3,0,1
3,4,0,0,35.0,1,0,53.1000,3,1,0
4,5,2,1,35.0,0,0,8.0500,3,0,1


Scaling

In [22]:
minmax = MinMaxScaler(feature_range=(0,1))
minmax.fit(X_train)
minmax.fit(X_test)
X_train = minmax.transform(X_train)
X_test = minmax.transform(X_test)

ML Algms

**RandomForest Classifier**

In [30]:
rand_fore = RandomForestClassifier()
rand_fore.fit(X_train, y_train)
y_predict = rand_fore.predict(X_test)
submission = pd.DataFrame({'PassengerId': titanic_test_data.PassengerId,
                           'Survived': y_predict})
submission.to_csv('/content/drive/My Drive/titanic.zip (Unzipped Files)/submission.csv', index=False)